When I merge the detected fire centroids into the fire perimeter boundary shapefiles, I end up with more observations than I started with (even doing a left join on the centroids), which suggests that there are some fire centroids that are matching on to multiple fire perimeters. I need to figure out why so that I can then decide what to do about it. 

In [12]:
import psycopg2
import numpy as np

In [13]:
# First we need to merge on the perimeter boundary files, and then figure out which detected
# fire centroids have merged on in mutiple places. In the detected fire data, a unique 
# index is (lat, long, date, gmt, src). We'll start with 2013. 
conn = psycopg2.connect('dbname=forest_fires')
cursor = conn.cursor()

cursor.execute('''CREATE TABLE merged_2013 AS
             (SELECT points.*, polys.fire_name, polys.fire, polys.agency, polys.unit_id
             FROM detected_fires_2013 as points
                    LEFT JOIN daily_fire_shapefiles_2013 as polys
             ON points.date = polys.date_ 
                AND ST_WITHIN(points.wkb_geometry, polys.wkb_geometry));
            ''')
conn.commit()

In [14]:
# Just to display what I'm talking about: 
cursor.execute('''SELECT COUNT(*)
                FROM detected_fires_2013;''')
print 'Detected_fires_2013 obs: ', cursor.fetchall()[0][0]

cursor.execute('''SELECT COUNT(*)
                FROM merged_2013;''')
print 'Merged_2013 obs: ', cursor.fetchall()[0][0]

Detected_fires_2013 obs:  184425
Merged_2013 obs:  207404


In [15]:
# Let's check if any obs. now have more than one row per (lat, long, date, gmt, and src), which
# prior to this merge was unique. 
cursor.execute('''SELECT COUNT(*) as totals
                FROM merged_2013
                GROUP BY lat, long, date, gmt, src
                ORDER BY totals DESC
                LIMIT 10;''')
cursor.fetchall()

[(6L,), (6L,), (6L,), (6L,), (6L,), (6L,), (6L,), (6L,), (6L,), (6L,)]

Okay, cool this is exactly what I thought. Let's get the (lat, long, date, gmt, src) of some 
of these obs and checkout what is going on. 

In [16]:
cursor.execute('''WITH totals_table AS 
                (SELECT COUNT(*) as totals, lat, long, date, gmt, src
                FROM merged_2013
                GROUP BY lat, long, date, gmt, src)
                
                SELECT lat, long, date, gmt, src 
                FROM totals_table 
                WHERE totals > 1;''')
duplicates_list = cursor.fetchall()

In [17]:
# Let's just go down the above list and figure out what is going on. 
duplicates_info = []
for duplicate in duplicates_list[:20]: 
    lat_coord, long_coord, date1, gmt, src = duplicate
    
    cursor.execute('''SELECT fire_name, fire, unit_id, agency
                    FROM merged_2013
                    WHERE lat = {} and long = {}
                        and gmt = {} and date = '{}'
                        and src = '{}'; '''.format(lat_coord, long_coord, gmt, date1, src))
    duplicates_info.append([cursor.fetchall(), duplicate])

In [18]:
for duplicate in duplicates_info[:10]: 
    print '-' * 50 
    print duplicate[0]
    print '\n' * 2
    print duplicate[1]

--------------------------------------------------
[('Rim', None, 'CA-STF', 'USFS'), ('Rim', None, 'CA-STF', 'USFS'), ('Rim', None, 'CA-STF', 'USFS')]



(Decimal('38.015'), Decimal('-119.955'), datetime.date(2013, 8, 26), Decimal('2101'), 'ssec')
--------------------------------------------------
[('Lake', None, 'ID-NPF', 'USFS'), ('Lake Complex', None, 'ID-NPF', 'USFS')]



(Decimal('45.569'), Decimal('-114.981'), datetime.date(2013, 8, 18), Decimal('1835'), 'gsfc')
--------------------------------------------------
[('Aspen', None, 'CA-SNF', 'USFS'), ('Aspen', None, 'CA-SNF', 'USFS')]



(Decimal('37.317'), Decimal('-119.279'), datetime.date(2013, 7, 26), Decimal('626'), 'rsac')
--------------------------------------------------
[('Tres Lagunas', None, 'NM-N4S', 'State Agency'), ('Tres Lagunas', None, 'NM-N4S', 'State Agency'), ('Tres Lagunas', None, 'NM-N4S', 'State Agency')]



(Decimal('35.711'), Decimal('-105.635'), datetime.date(2013, 6, 1), Decimal('442'), 'rsac')
------------

From the above, it doesn't look like I'll be able to tell too much, with the exception of how many perimeter boundaries that given (lat, long, date, gmt, src) merged onto. I think to solve this definitively, I need to graph some of the above boundaries and see how/if they overlap (I assume they must overlap, or else I wouldn't be having this issue). 